<a href="https://colab.research.google.com/github/AdarshShah/final-year-project/blob/master/project_sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [0]:
import math
import pandas as pd
import psycopg2
import getpass
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Input, GRU, Dense, LSTM, Flatten, Reshape, Lambda, CuDNNLSTM, Dropout
import matplotlib.pyplot as plt

# Declare Global Variables

In [0]:
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'
password = 'postgres'

table = dict()
standardScaler = StandardScaler()
minMaxScaler = MinMaxScaler()

tabs = ['uo','vitals','labs','ventsettings','vd','bgart','gcs']

columns = ['icustay_id', 'hadm_id','sepsis_explicit', 'septic_shock_explicit', 'severe_sepsis_explicit',
           'urineoutput','heartrate_min', 'heartrate_max','sysbp_min', 'sysbp_max', 'diasbp_min', 'diasbp_max', 'meanbp_min', 'meanbp_max', 'resprate_min',
           'resprate_max', 'tempc_min', 'tempc_max', 'spo2_min', 'spo2_max', 'glucose_min', 'glucose_max','aniongap_min', 'aniongap_max', 
           'albumin_min', 'albumin_max', 'bands_min', 'bands_max', 'bicarbonate_min', 'bicarbonate_max', 'bilirubin_min', 'bilirubin_max',
           'creatinine_min', 'creatinine_max', 'chloride_min', 'chloride_max', 'hematocrit_min', 'hematocrit_max', 
           'hemoglobin_min', 'hemoglobin_max', 'lactate_min', 'lactate_max', 'platelet_min', 'platelet_max', 'potassium_min', 'potassium_max',
           'ptt_min', 'ptt_max', 'inr_min', 'inr_max', 'pt_min', 'pt_max', 'sodium_min', 'sodium_max', 'bun_min', 'bun_max', 'wbc_min', 'wbc_max',
           'mechvent', 'extubated', 'selfextubated','ventnum','specimen_prob', 'so2', 'spo2', 'po2', 'pco2',
           'fio2_chartevents', 'fio2', 'aado2', 'aado2_calc', 'pao2fio2', 'ph','baseexcess', 'bicarbonate', 'totalco2', 'hematocrit', 'hemoglobin',
           'carboxyhemoglobin', 'methemoglobin', 'chloride', 'calcium', 'temperature', 'potassium', 'sodium', 'lactate', 'glucose', 'intubated',
           'tidalvolume', 'ventilationrate', 'ventilator', 'peep', 'o2flow', 'requiredo2','mingcs', 'gcsmotor',
           'gcsverbal', 'gcseyes', 'endotrachflag','specimen', 'specimen_pred']

# Create Database Connection

In [0]:
conn = psycopg2.connect(user=user,password=password,dbname=dbname,host=host)
cur = conn.cursor()
cur.execute('SET search_path to {}'.format(schema))

# Load Patient data




In [0]:
sepsis = pd.read_csv('/home/adarsh/Documents/sepsis/sepsis3-df.csv')
table['sepsis'] = sepsis

def get_pandas(table):
  query = "SELECT * FROM "+table+";"
  table = pd.read_sql(query,conn)
  return table

for tab in tabs:
  table[tab] = get_pandas(tab)

In [0]:
sepsis_dataset = table['sepsis']

for tab in tabs:
  print("\n%s:"%tab)
  f_key = []
  for i in sepsis_dataset.columns:
    if i in table[tab].columns:
      print(i,end=' ')
      f_key.append(i)
  b = table[tab]
  sepsis_dataset = pd.merge(sepsis_dataset,b,on=f_key,how='left')
  sepsis_dataset = sepsis_dataset.drop_duplicates() 

In [0]:
X = sepsis_dataset[columns]    

# Data Preprocessing

In [0]:
X = X.drop_duplicates()
X = X.fillna(value=X.mean())    
X = X.fillna(value=0) 

In [0]:
x = X.iloc[:,5:-2]

standardScaler = standardScaler.fit(x.values)
x = standardScaler.transform(x)

minMaxScaler = minMaxScaler.fit(x)
x = minMaxScaler.transform(x)

X.iloc[:,5:-2] = pd.DataFrame(x,columns=columns[5:-2])

In [0]:
x_train, x_test = train_test_split(X, test_size=0.2)

y_train = x_train.iloc[:,2:5]
x_train = x_train.iloc[:,5:]

y_test = x_test.iloc[:,2:5]
x_test = x_test.iloc[:,5:]